<a href="https://colab.research.google.com/github/itberrios/think_autonomous/blob/main/3d_reconstruction/MVS_Run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Multi View Stereo Notebook
Welcome to the Multi-View Stereo Notebook!
Here, you're going to learn how to build Multi-View Reconstruction algorithm from multiple images; such as a 360° video of an object!
<p>

**Your input will be a set of images, and your output will be a 3D Point Cloud.**

For example, you'll learn how to turn a series of images from this temple into a point cloud:
![img](https://perso.telecom-paristech.fr/boubek/papers/AMR/images/amr_scaled-window-matching.png)

Some of the elements of the "Rock" notebook, including the dataset, have been inspired from this fantastic repo: https://github.com/drewm1980/multi_view_stereo_benchmark

**A warning** — The topic is difficult, and it's likely that if you scatter the web, you won't find easy and perfectly working Python implementations of Multi-View Stereo algorithms. So *be patient*.

**However, if you understand how to do 2-View Reconstruction with Rectification and Disparity Estimation, then this should be straightforward.**

Now, let's get started!

How it works:
1.   Create the parameters for: **calibration, rectification, disparity estimation**
2.   Create a class that will **collect all parameters**.
3.   Load the **Images**
4.   Test **Rectification, Disparity, and Reprojection**
5.   Putting it **all together**


### 0 — Intro

#### Imports

In [ ]:
!wget https://stereo-vision.s3.eu-west-3.amazonaws.com/mvs.zip && unzip mvs.zip && rm mvs.zip

In [ ]:
!mv mvs/* . && rm -r mvs

In [ ]:
import numpy
import pathlib
from pathlib import Path
from time import time
import os
import inspect
pwd = os.path.dirname(os.path.abspath(inspect.stack()[0][1]))
from load_ply import save_ply
from load_camera_info import load_intrinsics, load_extrinsics
import cv2
import glob
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

In [ ]:
scenario = "rock" # rock or temple

#### Data Visualization

In [ ]:
images = sorted(glob.glob("data/"+scenario+"/undistorted/*.png"))

index = 10

plt.imshow(cv2.imread(images[index]))
plt.show()

In [ ]:
if scenario == "rock":
    images_cv = [cv2.rotate(cv2.cvtColor(cv2.imread(img), cv2.COLOR_BGR2RGB), cv2.ROTATE_180) for img in images]
    #images_cv = [cv2.cvtColor(cv2.imread(img), cv2.COLOR_BGR2RGB) for img in images]

elif scenario=="temple":
    #images_cv = [cv2.rotate(cv2.cvtColor(cv2.imread(img), cv2.COLOR_BGR2RGB), cv2.ROTATE_90_COUNTERCLOCKWISE) for img in images]
    images_cv = [cv2.cvtColor(cv2.imread(img), cv2.COLOR_BGR2RGB) for img in images]

plt.imshow(images_cv[index])
plt.show()

In [ ]:
f, ((ax0, ax1, ax2, ax3, ax4, ax5), (ax6, ax7, ax8, ax9, ax10, ax11)) = plt.subplots(2,6,figsize=(20,10))
ax0.imshow(images_cv[0])
ax1.imshow(images_cv[1])
ax2.imshow(images_cv[2])
ax3.imshow(images_cv[3])
ax4.imshow(images_cv[4])
ax5.imshow(images_cv[5])
ax6.imshow(images_cv[6])
ax7.imshow(images_cv[7])
ax8.imshow(images_cv[8])
ax9.imshow(images_cv[9])
ax10.imshow(images_cv[10])
ax11.imshow(images_cv[11])
plt.show()

In [ ]:
h, w, d = images_cv[index].shape

h, w, d

### 1 — Parameters
*   Topology
*   Rectification
*   Disparity Estimation
*   Camera Calibration

#### Topologies

In [ ]:
import collections
topologies = collections.OrderedDict()
topologies['360'] = tuple(zip((0,1,2,3,4,5,6,7,8,9,10,11),
                          (1,2,3,4,5,6,7,8,9,10,11,0)))

topologies['overlapping'] = tuple(zip((0,1,2,3,4,5,6,7,8,9,10),
                          (1,2,3,4,5,6,7,8,9,10,11)))

topologies['adjacent'] = tuple(zip((0,2,4,6,8,10),
                     (1,3,5,7,9,11)))
topologies['skipping_1'] = tuple(zip((0,3,6,9),
                 (1,4,7,10)))
topologies['skipping_2'] = tuple(zip((0,4,8),
                 (1,5,9)))


In [ ]:
for pair_index, (left_index,right_index) in enumerate(topologies["adjacent"]):
    print(left_index, right_index)

#### Camera Calibration & Rectification

In [ ]:
StereoRectifyOptions = {'imageSize':(w,h),
                        'flags':(0,cv2.CALIB_ZERO_DISPARITY)[0], # TODO explore other flags
                        'newImageSize':(w,h),
                        'alpha':0.5}

RemapOptions = {'interpolation':cv2.INTER_LINEAR}

CameraArrayOptions = {'channels':1,'num_cameras':12,'topology':'adjacent'}

#### Disparity Estimation

In [ ]:
StereoMatcherOptions = {'MinDisparity': -64, # Influences MAX depth
                        'NumDisparities': 256, # Influences MIN depth
                        'BlockSize': 21,
                        'SpeckleWindowSize': 0, # Must be strictly positive to turn on speckle post-filter.
                        'SpeckleRange': 0, # Must be >= 0 to enable speckle post-filter
                        'Disp12MaxDiff': 0}

StereoBMOptions = {
        'PreFilterType': (cv2.StereoBM_PREFILTER_NORMALIZED_RESPONSE, cv2.StereoBM_PREFILTER_XSOBEL)[0],
                   'PreFilterSize': 5, # preFilterSize must be odd and be within 5..255
                   'PreFilterCap': 63, # preFilterCap must be within 1..63. Used to truncate pixel values
                   'TextureThreshold': 10,
                   'UniquenessRatio': 10,
                   }

StereoSGBMOptions = {'PreFilterCap': 0,
                     'UniquenessRatio': 0,
                     'P1': 16*21*21, # "Depth Change Cost in Ensenso terminology"
                     'P2': 16*21*21, # "Depth Step Cost in Ensenso terminology"
                     'Mode': (cv2.StereoSGBM_MODE_SGBM, cv2.StereoSGBM_MODE_HH,
                              cv2.StereoSGBM_MODE_SGBM_3WAY)[1]}


FinalOptions = {'StereoRectify':StereoRectifyOptions,
        'StereoMatcher':StereoMatcherOptions,
        'StereoBM':StereoBMOptions,#change to "StereoSGBM" = StereoSGBMOptions if needed
        'CameraArray':CameraArrayOptions,
        'Remap':RemapOptions}

### 2 — Create a Class that will Collect all parameters
*   Init: Get parameters, get remap options, get rectification options, get Q, convert to global coordinates, get disparity matcher
*   Load images (part 3)
*   Run the other functions (part 4, 5)

#### Helper Function: Get the parameters

In [ ]:
def get_camera_parameters(options):
    flags=options['StereoRectify']['flags']
    distortion_coefficients = (0.0,0.0,0.0,0.0,0.0)
    left_distortion_coefficients = distortion_coefficients
    right_distortion_coefficients = distortion_coefficients
    imageSize = options['StereoRectify']['imageSize'] # w,h
    newImageSize = options['StereoRectify']['newImageSize']
    alpha = options['StereoRectify']['alpha']
    return left_distortion_coefficients, right_distortion_coefficients, imageSize, newImageSize, alpha

#### Helper Function: Calibrate & Rectify

In [ ]:
def calibrate_and_rectify(options, left_K, right_K, left_R, right_R, left_T, right_T):
    # Get the parameters
    left_distortion_coefficients, right_distortion_coefficients, imageSize, newImageSize, alpha = get_camera_parameters(options)

    # Stereo Rectify
    R_intercamera = numpy.dot(right_R, left_R.T) # R * T
    T_intercamera = right_T - numpy.dot(R_intercamera, left_T) # translation and rotation keeping the first one as baseline

    left_R_rectified, right_R_rectified, P1_rect, P2_rect, Q, validPixROI1, validPixROI2 = cv2.stereoRectify(
        cameraMatrix1 = left_K, distCoeffs1 = left_distortion_coefficients,
        cameraMatrix2 = right_K, distCoeffs2 = right_distortion_coefficients,
        imageSize=imageSize,
        newImageSize=newImageSize,
        R=R_intercamera, T=T_intercamera,
        flags=options['StereoRectify']['flags'] , alpha=alpha)

    # Back to Global
    R2,T2 = left_R, left_T # perspective is from left image.
    R3,T3 = R2.T,numpy.dot(-R2.T,T2) # Invert direction of transformation to map camera to world.
    R_left_rectified_to_global = numpy.dot(R3,left_R_rectified.T)
    T_left_rectified_to_global = T3
    extrinsics_left_rectified_to_global = R_left_rectified_to_global.astype(numpy.float32), T_left_rectified_to_global.astype(numpy.float32)

    # Create rectification maps
    rectification_map_type = cv2.CV_16SC2
    left_maps = cv2.initUndistortRectifyMap(left_K,
                                            left_distortion_coefficients,
                                            left_R_rectified,
                                            P1_rect,
                                            size=newImageSize,
                                            m1type=rectification_map_type)

    right_maps = cv2.initUndistortRectifyMap(right_K,
                                            right_distortion_coefficients,
                                            right_R_rectified,
                                            P2_rect,
                                            size=newImageSize,
                                            m1type=rectification_map_type)

    return Q, extrinsics_left_rectified_to_global, left_maps, right_maps

#### Helper Function: Get the Disparity Matching Algorithm

In [ ]:
def get_disparity_matcher(options):
    # Instantiate the matchers; they may do something slow internally...
    if 'StereoBM' in options:
        # Perform stereo matching using normal block matching
        numDisparities = options['StereoMatcher']['NumDisparities']
        blockSize = options['StereoMatcher']['BlockSize']
        matcher = cv2.StereoBM_create(numDisparities=numDisparities,blockSize=blockSize)
        setterOptions = {}
        setterOptions.update(options['StereoMatcher'])
        setterOptions.update(options['StereoBM'])
        for key,value in setterOptions.items():
            setter = eval('matcher.set'+key) # Returns the setter function
            setter(value) # Calls the setter function.

    elif 'StereoSGBM' in options:
        # Perform stereo matching using SGBM
        minDisparity = options['StereoMatcher']['MinDisparity']
        numDisparities = options['StereoMatcher']['NumDisparities']
        blockSize = options['StereoMatcher']['BlockSize']
        matcher = cv2.StereoSGBM_create(minDisparity=minDisparity,
                                    numDisparities=numDisparities,
                                    blockSize=blockSize)
        setterOptions = {}
        setterOptions.update(options['StereoMatcher'])
        setterOptions.update(options['StereoSGBM'])
        for key,value in setterOptions.items():
            setter = eval('matcher.set'+key) # Returns the setter function
            setter(value) # Calls the setter function.
    else:
        assert False, "Couldn't determine the matcher type from passed options!"

    return matcher

#### Class: Call all 3 helper functions

In [ ]:
from load_camera_info import load_all_camera_parameters

In [ ]:
class OpenCVStereoMatcher():
    def __init__(self,options=FinalOptions,calibration_path=None):
        self.options = options
        self.num_cameras = options['CameraArray']['num_cameras']
        self.topology = options['CameraArray']['topology']
        self.all_camera_parameters = load_all_camera_parameters(calibration_path)

        self.left_maps_array = []
        self.right_maps_array = []
        self.Q_array = []
        self.extrinsics_left_rectified_to_global_array = []

        for pair_index, (left_index,right_index) in enumerate(topologies[self.topology]):
            ## 1 — Get R, T, W, H for each camera
            left_K, left_R, left_T, left_width, left_height = [self.all_camera_parameters[left_index][key] for key in ('camera_matrix','R','T','image_width','image_height')]
            right_K, right_R, right_T, right_width, right_height = [self.all_camera_parameters[right_index][key] for key in ('camera_matrix','R','T','image_width','image_height')]
            h,w = left_height, left_width

            # 2 — Stereo Calibrate & Rectify
            Q, extrinsics_left_rectified_to_global, left_maps, right_maps = calibrate_and_rectify(options, left_K, right_K,
                                                                                                  left_R, right_R,
                                                                                                  left_T, right_T)
            self.Q_array.append(Q)
            self.extrinsics_left_rectified_to_global_array.append(extrinsics_left_rectified_to_global)
            self.left_maps_array.append(left_maps)
            self.right_maps_array.append(right_maps)

            # 3 — Get Matcher
            self.matcher = get_disparity_matcher(options)

In [ ]:
imagesPath = Path('data/'+scenario+'/undistorted')
workDirectory=Path('.')

opencv_matcher = OpenCVStereoMatcher(options=FinalOptions,calibration_path=imagesPath)

In [ ]:
### Print the DIsparity Matcher parameters etc ... to check everything is correct
print(opencv_matcher.Q_array[0])

### 3 — Load the Images (part of the class)

In [ ]:
def load_images(self,imagesPath):
    # Load a set of images from disk. Doesn't do processing yet.
    imagesPath = imagesPath.resolve()

    # Load the undistorted images off of disk
    print('Loading the images off of disk...')
    num_cameras = len(list(imagesPath.glob('*.png')))
    assert self.num_cameras == num_cameras, 'Mismatch in the number of available images!'
    images = []
    for i in range(num_cameras):
        fileName = 'image_camera%02i.png' % (i + 1)
        filePath = imagesPath / fileName
        print('Loading image',filePath)
        colorImage = cv2.imread(str(filePath))
        grayImage = cv2.cvtColor(colorImage, cv2.COLOR_BGR2GRAY)
        images.append(grayImage)
        expected_parameters = self.all_camera_parameters[i]
        w,h = expected_parameters['image_width'], expected_parameters['image_height']
        assert grayImage.shape == (h,w), 'Mismatch in image sizes!'
    self.images = images

OpenCVStereoMatcher.load_images = load_images

In [ ]:
opencv_matcher.load_images(imagesPath)

In [ ]:
plt.imshow(opencv_matcher.images[0])
plt.show()

### 4 — Test Parameters and Implement Reconstruction

The run() function will look like this:
1.   For each pair of the topology...
2.   Load the two images and rectification maps
3.   Rectify
4.   Load Disparity & Compute it
5.   Reproject to 3D, Postprocess, Save

In [ ]:
StereoMatcherOptions = {'MinDisparity': -64, # Influences MAX depth
                        'NumDisparities': 256, # Influences MIN depth
                        'BlockSize': 21,
                        'SpeckleWindowSize': 0, # Must be strictly positive to turn on speckle post-filter.
                        'SpeckleRange': 0, # Must be >= 0 to enable speckle post-filter
                        'Disp12MaxDiff': 0}

StereoBMOptions = {
        'PreFilterType': (cv2.StereoBM_PREFILTER_NORMALIZED_RESPONSE, cv2.StereoBM_PREFILTER_XSOBEL)[0],
                   'PreFilterSize': 5, # preFilterSize must be odd and be within 5..255
                   'PreFilterCap': 63, # preFilterCap must be within 1..63. Used to truncate pixel values
                   'TextureThreshold': 10,
                   'UniquenessRatio': 10,
                   }

StereoSGBMOptions = {'PreFilterCap': 0,
                     'UniquenessRatio': 0,
                     'P1': 16*21*21, # "Depth Change Cost in Ensenso terminology"
                     'P2': 16*21*21, # "Depth Step Cost in Ensenso terminology"
                     'Mode': (cv2.StereoSGBM_MODE_SGBM, cv2.StereoSGBM_MODE_HH,
                              cv2.StereoSGBM_MODE_SGBM_3WAY)[1]}

StereoRectifyOptions = {'imageSize':(w,h),
                        'flags':(0,cv2.CALIB_ZERO_DISPARITY)[0], # TODO explore other flags
                        'newImageSize':(w,h),
                        'alpha':0.5}

RemapOptions = {'interpolation':cv2.INTER_LINEAR}

CameraArrayOptions = {'channels':1,'num_cameras':12,'topology':'overlapping'} #topology=skipping_1, skipping_2, adjacent, overlapping, 360

FinalOptions = {'StereoRectify':StereoRectifyOptions,
        'StereoMatcher':StereoMatcherOptions,
        'StereoBM':StereoBMOptions,#change to "StereoSGBM" = StereoSGBMOptions if needed
        'CameraArray':CameraArrayOptions,
        'Remap':RemapOptions}

#### Step 1. Rectification
First, we're going to see if our **rectification** is correct.

The process for that is as follows:
1.   Load 2 images & Visualize them
2.   Load the rectification map
3.   Rectify & Show result

In [ ]:
opencv_matcher = OpenCVStereoMatcher(options=FinalOptions,calibration_path=imagesPath)
opencv_matcher.load_images(imagesPath)

# Get the Images
img0 = opencv_matcher.images[0]
img1 = opencv_matcher.images[1]

# Get the Maps
left_maps = opencv_matcher.left_maps_array[0]
right_maps = opencv_matcher.right_maps_array[0]

# Rectify with the parameters
remap_int = opencv_matcher.options['Remap']['interpolation']
left_image_rectified = cv2.remap(img0, left_maps[0],left_maps[1], remap_int)
right_image_rectified = cv2.remap(img1, right_maps[0], right_maps[1], remap_int)

# Show Results
f, (f0, f1, f2) = plt.subplots(1,3,figsize=(20,10))
f0.imshow(img0)
f1.imshow(left_maps[1])
f2.imshow(left_image_rectified)
f, (f3, f4, f5) = plt.subplots(1,3,figsize=(20,10))
f3.imshow(img1)
f4.imshow(right_maps[1])
f5.imshow(right_image_rectified)
plt.show()

In [ ]:
plt.imshow(abs(left_image_rectified - right_image_rectified));

#### Step 2: Disparity Estimation
Now, we're going to play with the disparity and see if we can do something interesting. This time, and to simplify the code, **we won't do it for every single image of the topology, just 2**, and we'll generalize in our final code.

We're just goint to grab the last 2 images we tried our code on. — make sure your topology allows for disparity estimate.

In [ ]:
# Find Disparity
matcher = opencv_matcher.matcher
disparity_img = matcher.compute(left_image_rectified, right_image_rectified)

if disparity_img.dtype == numpy.int16:
    disparity_img = disparity_img.astype(numpy.float32)
    disparity_img /= 16

# Show Results
plt.imshow(disparity_img)
plt.show()

#### 2Bis — Find the right parameters

In [ ]:
from ipywidgets import interact, interactive, fixed

def compute_disparity(image, img_pair, num_disparities=6*16, block_size=11, window_size=6, uniqueness_ratio=0, speckleWindowSize = 200, matcher="stereo_sgbm", show_disparity=True):
    if matcher == "stereo_bm":
        new_image = cv2.StereoBM_create(numDisparities=num_disparities,blockSize=block_size)
        new_image.setPreFilterType(1)
        new_image.setUniquenessRatio(uniqueness_ratio)
        new_image.setSpeckleRange(2)
        new_image.setSpeckleWindowSize(speckleWindowSize)
    elif matcher == "stereo_sgbm":
        new_image = cv2.StereoSGBM_create(minDisparity=0, numDisparities=num_disparities, blockSize=block_size,
                                         uniquenessRatio=uniqueness_ratio, speckleWindowSize=speckleWindowSize, speckleRange=2, disp12MaxDiff=1,
                                         P1=8 * 1 * window_size **2, P2=32 * 1 * window_size **2)

    new_image = new_image.compute(image, img_pair).astype(numpy.float32)/16

    if (show_disparity==True):
        plt.figure(figsize = (20,10))
        plt.imshow(new_image, cmap="plasma")
        plt.show()
    return new_image


In [ ]:
num_d = (0,512,16)
b_s = (1,31,2)
window_s = (1,13,2)
uniqueness_r = (0, 10, 1)
speckle_w = (0, 250, 50)

disparity_left = interactive(compute_disparity, image=fixed(left_image_rectified), img_pair = fixed(right_image_rectified), num_disparities=num_d, block_size=b_s, window_size=window_s, matcher=["stereo_sgbm", "stereo_bm"], uniqueness_ratio= uniqueness_r, speckleWindowSize=speckle_w)
display(disparity_left)

Once you have good parameters, be ready to recompute the **FinalOptions** variable

#### 2BisBis — CreSTEREO Disparity

In [ ]:
!git clone https://github.com/ibaiGorordo/ONNX-CREStereo-Depth-Estimation
! pip3 install -r ONNX-CREStereo-Depth-Estimation/requirements.txt
!pip3 install onnxruntime-gpu
import os
os.chdir("ONNX-CREStereo-Depth-Estimation")
from crestereo import CREStereo
iters = 20            # Lower iterations are faster, but will lower detail.
		             # Options: 2, 5, 10, 20

shape = (480, 640)   # Input resolution.
				     # Options: (240,320), (320,480), (380, 480), (360, 640), (480,640), (720, 1280)

version = "combined" # The combined version does 2 passes, one to get an initial estimation and a second one to refine it.
					 # Options: "init", "combined"

!git clone https://github.com/PINTO0309/PINTO_model_zoo.git
!bash PINTO_model_zoo/284_CREStereo/download_iter20.sh
!ls
!cp -r crestereo_*.onnx models/
# Initialize model
model_path = f'models/crestereo_{version}_iter{20}_{shape[0]}x{shape[1]}.onnx'
depth_estimator = CREStereo(model_path)

In [ ]:
disparity_img = depth_estimator(left_image_rectified,right_image_rectified)
plt.imshow(disparity_img)
plt.show()

In [ ]:
disparity_img.min(), disparity_img.max()

In [ ]:
plt.imshow(disparity_img >= 100)

#### Step 3: Project to 3D
Finally, we'll see the code to project into 3D.
This code will mainly project the Disparity into the Q matrix.

In [ ]:
# Reproject 3D
Q = opencv_matcher.Q_array[0]
threedeeimage = cv2.reprojectImageTo3D(disparity_img, Q, handleMissingValues=True,ddepth=cv2.CV_32F)
threedeeimage = numpy.array(threedeeimage)

# Postprocess
xyz = threedeeimage.reshape((-1,3)) # x,y,z now in three columns, in left rectified camera coordinates
z = xyz[:,2]
goodz = z < 9999.0
xyz_filtered = xyz[goodz,:]

# Global Coordinates
R_left_rectified_to_global, T_left_rectified_to_global = opencv_matcher.extrinsics_left_rectified_to_global_array[0]
xyz_global_ = numpy.dot(xyz_filtered, R_left_rectified_to_global.T) + T_left_rectified_to_global.T

# Save PLY
save_ply(xyz_global_, 'pair_test_0_1.ply')

In [ ]:
!pip install pyntcloud
from pyntcloud import PyntCloud

In [ ]:
object_3d = PyntCloud.from_file("pair_test_0_1.ply")
object_3d.plot()

### 5 — Putting it all together

In [ ]:
def run(self):
    assert self.all_camera_parameters is not None, 'Camera parameters not loaded yet; You should run load_all_camera_parameters first!'
    xyz_global_array = [None]*len(topologies[self.topology])

    def run_pair(pair_idx, left_idx, right_idx):
        # Load the proper images and rectification maps
        left_img, right_img = self.images[left_idx], self.images[right_idx]
        left_maps = self.left_maps_array[pair_idx]
        right_maps = self.right_maps_array[pair_idx]

        # Rectify
        remap_interpolation = self.options['Remap']['interpolation']
        left_image_rectified = cv2.remap(left_img, left_maps[0],left_maps[1], remap_interpolation)
        right_image_rectified = cv2.remap(right_img, right_maps[0], right_maps[1], remap_interpolation)

        # Load & Find Disparity
        disparity_image = self.matcher.compute(left_image_rectified, right_image_rectified)

        if disparity_image.dtype == numpy.int16:
            disparity_image = disparity_image.astype(numpy.float32)
            disparity_image /= 16

        plt.imshow(disparity_image)
        plt.show()

        # Reproject 3D
        Q = self.Q_array[pair_idx]
        threedeeimage = cv2.reprojectImageTo3D(disparity_image, Q, handleMissingValues=True,ddepth=cv2.CV_32F)
        threedeeimage = numpy.array(threedeeimage)

        # Postprocess
        xyz = threedeeimage.reshape((-1,3)) # x,y,z now in three columns, in left rectified camera coordinates
        z = xyz[:,2]
        goodz = z < 9999.0
        xyz_filtered = xyz[goodz,:]

        # Global Coordinates
        R_left_rectified_to_global, T_left_rectified_to_global = self.extrinsics_left_rectified_to_global_array[pair_idx]
        xyz_global = numpy.dot(xyz_filtered, R_left_rectified_to_global.T) + T_left_rectified_to_global.T

        # Save PLY
        save_ply(xyz_global, 'pair_'+str(left_index)+'_'+str(right_index)+'.ply')
        xyz_global_array[pair_index] = xyz_global

    for pair_index, (left_index,right_index) in enumerate(topologies[self.topology]):
        run_pair(pair_index, left_index, right_index)

    xyz = numpy.vstack(xyz_global_array)
    return xyz

OpenCVStereoMatcher.run = run

### Main

In [ ]:
imagesPath = Path('/content/data/'+scenario+'/undistorted')
workDirectory=Path('.')

opencv_matcher = OpenCVStereoMatcher(options=FinalOptions, calibration_path=imagesPath)
opencv_matcher.load_images(imagesPath)
xyz = opencv_matcher.run()

save_ply(xyz, "therock.ply")

In [ ]:
%ls

In [ ]:
object_3d = PyntCloud.from_file("therock.ply")
object_3d.plot()

# [Temple Reconstruction] The Temple is Under Attack!

![](https://vision.middlebury.edu/mview/data/images/temple0119.jpg)

Dear warrior,

**Our beloved temple is under attack.**

The enemy is at the door, and they're going to kill the younglings.
We can hold the siege, but soon or later, it's going to fall.

Only you can preserve its memory.

We took 12 pictures of this temple, and saved the camera information in a folder named "temple".

**Everything is done, but we need to adjust the Stereo Vision parameters.**

And only you can do it.

Help us, you're our only hope.

<p>

Below are a few functions to help you with the task!

#### 1. First, the **"load camera parameters"** function has been modified to handle the new dataset. It's already imported with the wget function you ran (cell #1 of this notebook):

In [ ]:
from load_camera_info_temple import load_all_camera_parameters_temple

#### 2. Then, you'll need to load the correct images, with the right scenario.

In [ ]:
scenario ="temple"

####3. These are some parameters that have been implemented by the team working on Temple Reconstruction in C++. Here is the original repo where you can grab TONS of cool ideas: https://github.com/temple-reconstruction/mview/blob/master/pure-cv/main.cpp

In [ ]:
StereoMatcherTemple = {
                        'MinDisparity': 0,
                        'NumDisparities': 64,
                        'BlockSize': 7,
                        'Disp12MaxDiff': 0,
                        'PreFilterCap' : 0,
                        'UniquenessRatio' : 15,
                        'SpeckleWindowSize' : 50,
                        'SpeckleRange' : 1
                     }

StereoSGBMTemple = {
    'PreFilterCap': 0,
    'UniquenessRatio': 0,
    'P1': 8, # "Depth Change Cost in Ensenso terminology"
    'P2': 32, # "Depth Step Cost in Ensenso terminology"
    }

TempleOptions = {'StereoRectify':StereoRectifyOptions,
        'StereoMatcher':StereoMatcherTemple,
        'StereoSGBM':StereoSGBMTemple,#change to "StereoSGBM" = StereoSGBMOptions if needed
        'CameraArray':CameraArrayOptions,
        'Remap':RemapOptions}

####4. In the notebook, we have a get_disparity() function — This one is doing it with the new options and implements StereoSGBM instead.

In [ ]:
def get_disparity_temple(options):
    minDisparity = options['StereoMatcher']['MinDisparity']
    numDisparities = options['StereoMatcher']['NumDisparities']
    blockSize = options['StereoMatcher']['BlockSize']
    P1 = options['StereoSGBM']['P1']
    P2 = options['StereoSGBM']['P2']
    Disp12MaxDiff = options['StereoMatcher']['Disp12MaxDiff']
    uniquenessRatio = options['StereoSGBM']['UniquenessRatio']
    speckleWindowSize = options['StereoMatcher']['SpeckleWindowSize']
    speckleRange = options['StereoMatcher']['SpeckleRange']
    PreFilterCap = options['StereoSGBM']['PreFilterCap']

    matcher = cv2.StereoSGBM_create(
                                    minDisparity=minDisparity,
                                    numDisparities=numDisparities,
                                    blockSize=blockSize,
                                    P1=P1,
                                    P2=P2,
                                    disp12MaxDiff = Disp12MaxDiff,
                                    preFilterCap =PreFilterCap,
                                    uniquenessRatio=uniquenessRatio,
                                    speckleWindowSize=speckleWindowSize,
                                    speckleRange=speckleRange)
    setterOptions = {}
    setterOptions.update(options['StereoMatcher'])
    setterOptions.update(options['StereoSGBM'])

    #setterOptions.update(options['StereoSGBM'])
    for key,value in setterOptions.items():
        setter = eval('matcher.set'+key) # Returns the setter function
        setter(value) # Calls the setter function.

    return matcher

####5. Following the C++ folks, this function implements disparity estimation with StereoSGBM but also adds something called a WLS filter.

Here is an image without the WLS filter:
![](https://docs.opencv.org/3.4/ambush_5_bm.png)
And one with the filter:
![](https://docs.opencv.org/3.4/ambush_5_bm_with_filter.png)


In [ ]:
def get_disparity_temple_filter(rimg1, rimg2):
    maxd = 2
    window_size = 5
    left_matcher = cv2.StereoSGBM_create(minDisparity=-maxd, numDisparities=11,
        blockSize=5,
        P1=8 * 3 * window_size ** 2,
        P2=32 * 3 * window_size ** 2,
        disp12MaxDiff=1,
        uniquenessRatio=15,
        speckleWindowSize=0,
        speckleRange=2,
        preFilterCap=63,
        mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY
    )
    right_matcher = cv2.ximgproc.createRightMatcher(left_matcher)
    lmbda = 8000
    sigma = 1.5
    wls_filter = cv2.ximgproc.createDisparityWLSFilter(matcher_left=left_matcher)
    wls_filter.setLambda(lmbda)
    wls_filter.setSigmaColor(sigma)
    displ = left_matcher.compute(rimg1, rimg2)
    dispr = right_matcher.compute(rimg2, rimg1)
    displ = numpy.int16(displ)
    dispr = numpy.int16(dispr)
    disparity = wls_filter.filter(displ, rimg1, None, dispr) / 16.0
    return disparity

####6. In the code, you'll need to call the right disparity function.

In [ ]:
imagesPath = Path('data/'+scenario+'/undistorted')

opencv_matcher = OpenCVStereoMatcher(options=TempleOptions,calibration_path=imagesPath)
opencv_matcher.load_images(imagesPath)

# Get the Images
img0 = opencv_matcher.images[0]
img1 = opencv_matcher.images[1]

# Get the Maps
left_maps = opencv_matcher.left_maps_array[0]
right_maps = opencv_matcher.right_maps_array[0]

# Rectify with the parameters
remap_int = opencv_matcher.options['Remap']['interpolation']
left_image_rectified = cv2.remap(img0, left_maps[0],left_maps[1], remap_int)
right_image_rectified = cv2.remap(img1, right_maps[0], right_maps[1], remap_int)

In [ ]:
# Show Results
f, (f0, f1, f2) = plt.subplots(1,3,figsize=(20,10))
f0.imshow(img0)
f1.imshow(left_maps[1])
f2.imshow(left_image_rectified)
f, (f3, f4, f5) = plt.subplots(1,3,figsize=(20,10))
f3.imshow(img1)
f4.imshow(right_maps[1])
f5.imshow(right_image_rectified)
plt.show()

Alright, good luck. If you get good results, come to me with the solutions. I'll have something cool for you...

In [ ]:
disparity_img = get_disparity_temple_filter(left_image_rectified, right_image_rectified).astype(numpy.float32)/16

In [ ]:
plt.imshow(img0);

In [ ]:
plt.imshow(img1)

In [ ]:
left_image_rectified.min(), left_image_rectified.max()

In [ ]:
plt.imshow(left_image_rectified)